# Churn Prediction


Run the Mlflow UI:

```sh
uv run --with mlflow==3.7.0 mlflow ui --backend-store-uri sqlite:///mlflow.db
```


Install packages


In [1]:
!uv pip install -q \
    requests==2.32.5

Append notebooks directory to sys.path


In [ ]:
import sys

sys.path.append("../../../..")

Import packages


In [ ]:
import requests

Training pipeline

```py title='app/train.py'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/train.py"
```


Run pipeline


In [4]:
!cd app && \
    uv run train.py

2026/01/16 18:22:31 INFO mlflow.tracking.fluent: Experiment with name 'telco-churn-prediction' does not exist. Creating a new experiment.
2026/01/16 18:22:45 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run enthused-wasp-823 at: http://127.0.0.1:5000/#/experiments/1/runs/5385b8f560694da2b6db3b04a1bff394
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Application code

```py title='app/main.py'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/main.py"
```

Dockerfile

```dockerfile title='app/Dockerfile'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/Dockerfile"
```

Dependencies

```toml title='app/pyproject.toml'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/pyproject.toml"
```


In [5]:
!aws --endpoint-url=http://localhost:4566 s3 ls s3://mlflow-artifacts/ --recursive

2026-01-16 18:22:46        652 1/models/m-eeba80cd1ff2478b97fbc45fe5bd1294/artifacts/MLmodel
2026-01-16 18:22:46        222 1/models/m-eeba80cd1ff2478b97fbc45fe5bd1294/artifacts/conda.yaml
2026-01-16 18:22:46       2601 1/models/m-eeba80cd1ff2478b97fbc45fe5bd1294/artifacts/model.pkl
2026-01-16 18:22:46         98 1/models/m-eeba80cd1ff2478b97fbc45fe5bd1294/artifacts/python_env.yaml
2026-01-16 18:22:46        109 1/models/m-eeba80cd1ff2478b97fbc45fe5bd1294/artifacts/requirements.txt


Build docker image


In [10]:
!cd app && \
    docker build -q -t fastapi-predict-churn-mlflow .

sha256:8efad11ce0ac3b0a66539178811863885cc9320d29b350ae14ef99e322b5c795


Run application


In [ ]:
!docker run --rm -it \
  --network datadev_datadev-net \
  -d \
  -p 9696:9696 \
  -e MLFLOW_TRACKING_URI=http://mlflow:5000 \
  -e MLFLOW_S3_ENDPOINT_URL=http://localstack:4566 \
  -e AWS_ACCESS_KEY_ID=test \
  -e AWS_SECRET_ACCESS_KEY=test \
  -e AWS_DEFAULT_REGION=us-east-1 \
  -e AWS_EC2_METADATA_DISABLED=true \
  -e RUN_ID=eeba80cd1ff2478b97fbc45fe5bd1294 \
  fastapi-predict-churn-mlflow

Check running applications


In [12]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS                                                                  NAMES
917d7dd1e9ef   fastapi-predict-churn-mlflow   "uvicorn main:app --…"   34 seconds ago   Up 33 seconds               0.0.0.0:9696->9696/tcp, [::]:9696->9696/tcp                            practical_thompson
a9bb4ca672cc   datadev-mlflow                 "mlflow server --hos…"   18 minutes ago   Up 18 minutes               0.0.0.0:5000->5000/tcp, [::]:5000->5000/tcp                            mlflow
78fca0cbd827   localstack/localstack:latest   "docker-entrypoint.sh"   18 minutes ago   Up 18 minutes (unhealthy)   4510-4559/tcp, 5678/tcp, 0.0.0.0:4566->4566/tcp, [::]:4566->4566/tcp   localstack
7c3f39be4336   postgres:18-alpine3.22         "docker-entrypoint.s…"   18 minutes ago   Up 18 minutes               0.0.0.0:5432->5432/tcp, [::]:5432->5432/tcp                            mlflow-postgres


Request application


In [ ]:
url = "http://localhost:9696/predict"

customer = {
    "gender": "female",
    "seniorcitizen": 0,
    "partner": "yes",
    "dependents": "no",
    "phoneservice": "no",
    "multiplelines": "no_phone_service",
    "internetservice": "dsl",
    "onlinesecurity": "no",
    "onlinebackup": "yes",
    "deviceprotection": "no",
    "techsupport": "no",
    "streamingtv": "no",
    "streamingmovies": "no",
    "contract": "month-to-month",
    "paperlessbilling": "yes",
    "paymentmethod": "electronic_check",
    "tenure": 1,
    "monthlycharges": 29.85,
    "totalcharges": 29.85,
}

response = requests.post(url, json=customer)

predictions = response.json()

print(predictions)

{'churn_probability': 0.6638167617162171, 'churn': True}


Stop container


In [13]:
!docker stop 917d7dd1e9ef

917d7dd1e9ef


Delete container image


In [15]:
!docker rmi fastapi-predict-churn-mlflow

Untagged: fastapi-predict-churn-mlflow:latest
Deleted: sha256:8efad11ce0ac3b0a66539178811863885cc9320d29b350ae14ef99e322b5c795
